Treat image ROI takes electron diffraction patterns and calibrate the image, rotate it, draw the BZ around the Bragg peak and average equivalent BZ together

In [ ]:

%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# plt.rc('font', family='Latin Modern Roman',size=26, weight = 'bold')
# plt.rc('text', usetex=True)
# plt.rcParams['text.latex.preamble'] = r'\boldmath'

# plt.rcParams["font.weight"] = "bold"

from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import cv2  # importing cv
import find_peak
import scipy.special as special
from matplotlib import ticker
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import nfft
import scipy
import scipy.fft
import functions_analysis as fa

%matplotlib inline 


# scan_number = ['r471', 'r475', 'r470']
# scan_name = [r'0.8 mJ/cm$^2$', r'1.27 mJ/cm$^2$', r'1.6 mJ/cm$^2$']

scan_number = ['r464', 'r471']
scan_name = ['800nm', '400nm']

path_main =r'C:\Users\rclaude\Documents\projet\graphite\drawing\UED_fig'
path=[]
df_list = []
imgON=[]
imgOFF=[]
delay=[]
# path_upload = path_main+r'\comparison\paper_transient'
for scan in scan_number:
    path.append(path_main  + r'\data\PROC_'+scan)
    df = pd.read_pickle(path_main + r'\data\PROC_'+scan, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
    imgOFF.append(np.stack(df['imgOFF']))
    imgON.append(np.stack(df['imgON']))
    delay.append(np.stack(df['delay']))
    df_list.append(df)

print(delay)


In [3]:
cc = []
BP_pos = []
BP_name = []
BP_name = ['{100}', '{200}', '{110}']
DS_pos = []
DS_name = [r'$\Gamma \to K$ 1o', r'$\Gamma \to K$ 2o', r'$\Gamma \to M$ 1o', r'$\Gamma \to M$ 2o']
DS_name = [r'$K$', r'$\Gamma \to M$']

Zorder=[]
cal = []
for df in df_list:
    L = df.attrs['peak_dist']
    Zo = df.attrs['Zorder_pos']
    Zorder.append(Zo)
    cal.append(df.attrs['calibration'])
    t0 = df.attrs['t0_index']

# cc = L2/2 * np.exp(1j * np.pi * ( 1./2 * np.arange(5)))## for squared BZ
    cc.append( L * np.sqrt(3)/3 *np.exp(1j * np.pi  * ( 1./3 * np.arange(7)))) ## for hexagonal BZ
    #### construction position of the Bragg peak, ordered by family

    BP_pos.append(fa.bragg_peak_position(Zo, L))

    #### construction position of the K-pont, ordered by family
    DS_pos.append(fa.diffuse_scattering_position2(Zo, L))


In [ ]:
at=t0
bt=t0

lim=7.5
sens = .01
sens2 = 5e5

fig, axis = plt.subplots(len(imgON), 2, figsize=(18, 9*len(imgON)),layout='tight')
for o in range(len(imgON)):
     s1 = np.shape(imgON[o])[2]
     s2 = np.shape(imgON[o])[1]
     print('before time 0:', delay[o][:bt])
     print('after time 0:', delay[o][at:])
     ext = (cal[o]*(Zorder[o][1]), cal[o]*(Zorder[o][1]-s1), cal[o]*(Zorder[o][0]-s2), cal[o]*(Zorder[o][0]))
     img = (np.nanmean(imgON[o][at:], axis=0))/np.nanmean(imgOFF[o][at:], axis=0) - (np.nanmean(imgON[o][:bt], axis=0))/np.nanmean(imgOFF[o][:bt], axis=0)

     im1= axis[o][0].imshow(imgON[o][0], cmap='bwr',  extent = ext ,vmin=-0, vmax=+sens2, aspect='equal')
     axis[o][0].set_title(scan_number[o] + ': ' + scan_name[o])
     axis[o][0].set_xlabel(r'$k_x (A^{-1})$', fontweight='semibold')
     axis[o][0].set_ylabel(r'$k_y (A^{-1})$')
     axis[o][0].set_xlim(-lim,lim)
     axis[o][0].set_ylim(-lim,lim)

     im2 = axis[o][1].imshow(img, cmap='bwr', extent =ext ,vmin=-sens, vmax=sens, aspect='equal')
     axis[o][1].set_title(r'ON/OFF($t>$ 0 ps)')
     axis[o][1].set_xlabel(r'$k_x (A^{-1})$')
     axis[o][1].set_ylabel(r'$k_y (A^{-1})$')
     axis[o][1].set_xlim(-lim,lim)
     axis[o][1].set_ylim(-lim,lim)

     divider = make_axes_locatable(axis[o][0])
     cax = divider.append_axes('left', size='5%', pad=0.6)        
     fig.colorbar(im1, cax=cax, shrink=0.6, orientation='vertical', label = r'electron count', format=ticker.FuncFormatter(fa.fmt), location='left')

     divider = make_axes_locatable(axis[o][1])
     cax = divider.append_axes('right', size='5%', pad=0.6)        
     fig.colorbar(im2, cax=cax, shrink=0.6, orientation='vertical', label = r'electron count variation', format='%.3f', location='right')
     for i in range(len(BP_pos[o])):
          for j in range(len(BP_pos[o][i])):
               axis[o][0].add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
                    lw=0.5, edgecolor='k', clip_on=False, facecolor='none'))
               # axis[j][0].add_patch(Circle((cal*(Zorder[j]-BP_roi[j][i][1]), cal*(Zorder[j]-BP_roi[j][i][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
               axis[o][1].add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
                    lw=0.2, edgecolor='k', clip_on=False, facecolor='none'))
               # axis[1].add_patch(Circle((cal*(Zorder[1]-BP_roi[i][1]), cal*(Zorder[0]-BP_roi[i][0])), 4*cal, fill = False, edgecolor = 'k', lw = 0.5))

     for i in range(len(DS_pos[o])):
          for j in range(len(DS_pos[o][i])):
               axis[o][1].add_patch(Circle((cal[o]*(Zorder[o][1]-DS_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-DS_pos[o][i][j][0])),10*cal[o], fill = False, edgecolor = 'k', lw = 0.5))

# plt.savefig(path_upload +r'\overall.png', format='png', dpi=300)

In [ ]:
### 2d voigt

o=1
print(BP_pos[o][0][0])
ROI = []
for o in [0,1]:
    ROI.append(find_peak.ROI_around(BP_pos[o][0][0], 15))
# ROI = find_peak.ROI_around(Zorder, 15)
rot=False
result_off_all = []
result_on_all = []
for o in [0,1]:
    result_on = []
    for imON in imgON[o]:
        result_on.append(find_peak.fit_2dvoigt(imON, ROI[o], show_plot=False, rotation=rot))
    result_on_all.append(result_on)
    result_off = [] 
    for imOFF in imgOFF[o]:
        result_off.append(find_peak.fit_2dvoigt(imOFF, ROI[o], show_plot=False, rotation=rot))
    result_off_all.append(result_off)

print(result_on[0])

In [6]:
def extract_parameter(param_list, param_name):
    return np.array([param[param_name].value for param in param_list])

In [ ]:
fig, axis = plt.subplots(2,4, figsize=(24, 12),layout='tight')

end = len(delay)
c_bp = ['k', 'orange', 'g']
c_bp = ['k', 'orange', 'g', 'b', 'm', 'sienna']

c_ds = ['b', 'm', 'y', 'sienna']
fig.suptitle(scan_number)

ax = axis[0][0]
ax.plot(delay[o], extract_parameter(result_on, 'amplitude')/extract_parameter(result_on, 'amplitude')[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, 'amplitude')/extract_parameter(result_off, 'amplitude')[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, 'amplitude')/extract_parameter(result_off, 'amplitude'), 'o-',   c='k',label='on/off')
ax.set_title('amplitude')


ax = axis[1][0]
par = 'offset'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

ax = axis[0][1]
par = 'centerx'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

ax = axis[1][1]
par = 'centery'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

ax = axis[0][2]
par = 'sigmax'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

ax = axis[1][2]
par = 'sigmay'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)


ax = axis[0][3]
par = 'gammax'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

ax = axis[1][3]
par = 'gammay'
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_on, par)[0], '+--',   c='r',label='on')
ax.plot(delay[o], extract_parameter(result_off, par)-extract_parameter(result_off, par)[0], '+--',   c='b',label='off')
ax.plot(delay[o], extract_parameter(result_on, par)-extract_parameter(result_off, par), 'o-',   c='k',label='on/off')
ax.set_title(par)

for ax in axis:
    for a in ax:
        a.legend()
        a.set_xlabel('delay (ps)')

In [ ]:
def calculate_fwhm_voigt2(sigma_x, sigma_y, gamma_x, gamma_y):
    # Gaussian FWHM
    fwhm_gaussian_x = 2 * np.sqrt(2 * np.log(2)) * sigma_x
    fwhm_gaussian_y = 2 * np.sqrt(2 * np.log(2)) * sigma_y
    
    # Lorentzian FWHM
    fwhm_lorentzian_x = 2 * gamma_x
    fwhm_lorentzian_y = 2 * gamma_y
    
    # Voigt FWHM
    fwhm_voigt_x = (0.5346 * fwhm_lorentzian_x +
                    np.sqrt(0.2166 * fwhm_lorentzian_x**2 + fwhm_gaussian_x**2))
    fwhm_voigt_y = (0.5346 * fwhm_lorentzian_y +
                    np.sqrt(0.2166 * fwhm_lorentzian_y**2 + fwhm_gaussian_y**2))
    
    return fwhm_voigt_x, fwhm_voigt_y
FWHM = {}
FWHM[scan_name[0]] = {}
FWHM[scan_name[1]] = {}


for o, result_on in enumerate(result_on_all):
    f_x_on, f_y_on = (calculate_fwhm_voigt2(extract_parameter(result_on, 'sigmax'), extract_parameter(result_on, 'sigmay'), extract_parameter(result_on, 'gammax'), extract_parameter(result_on, 'gammay')))
    FWHM[scan_name[o]]['f_x_on'] = f_x_on
    FWHM[scan_name[o]]['f_y_on'] = f_y_on

for o, result_off in enumerate(result_off_all):
    f_x_off, f_y_off = (calculate_fwhm_voigt2(extract_parameter(result_off, 'sigmax'), extract_parameter(result_off, 'sigmay'), extract_parameter(result_off, 'gammax'), extract_parameter(result_off, 'gammay')))
    FWHM[scan_name[o]]['f_x_off'] = f_x_off
    FWHM[scan_name[o]]['f_y_off'] = f_y_off

print(f_y_off)

In [9]:
plt.rcParams.update({'font.size': 20})
plt.rc('axes', linewidth=2)
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1
plt.rc('axes', linewidth=2)
# plt.rc('font', weight='semibold')
# plt.rcParams['text.latex.preamble'] = r'\usepackage{sfmath} \boldmath'
plt.rc('font', family='arial',size=26)#, weight='bold')
plt.rc('text', usetex=False)

In [10]:
scan_name = ['800nm', '400nm']
c = ['r', 'b']
sname = ['1.55 eV', '3.1 eV']


In [ ]:

fig, axis = plt.subplots(1,2, figsize=(12, 6),layout='tight')

end = len(delay)
c_bp = ['k', 'orange', 'g']
c_ds = ['b', 'm', 'y', 'sienna']
s = ['*-', 'o-', '+-']
leg_bp = []
leg_scan = []


# axis[0].plot(delay[o], f_x_on-f_x_on[0], label='x on')
# axis[0].plot(delay[o], f_x_off-f_x_off[0], label='x off')
for o in [0,1]:
    y = FWHM[scan_name[o]]['f_x_on']-FWHM[scan_name[o]]['f_x_off']
    axis[0].plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o], label=sname[o])
    # axis[0].errorbar(delay[o], y-y[0], yerr=FWHM[scan_name[o]]['error_x'], linestyle='-', marker='o', c=c[o], ecolor=c[o], label=sname[o])

axis[0].set_title(r'along $q_x$')

for o in [0,1]:
    y=FWHM[scan_name[o]]['f_y_on']-FWHM[scan_name[o]]['f_y_off']
    axis[1].plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o] ,label=sname[o])
axis[1].set_title(r'along $q_y$')


for a in axis:
    a.set_ylabel('$\Delta$FHWM ($10^{-4}/\AA$)')
    a.set_xlabel('delay (ps)')
    a.legend()
    a.tick_params(axis='both', direction='in', pad = 15)
    a.set_xlim(-3, 10)
    a.set_ylim(-4,1)


# for i in range(len(height)):
#     axis[2].plot(delay[i], height[i][:,0]-height_off[i][:,0], label=scan_name[i]+' vert height')
#     axis[2].plot(delay[i], height[i][:,1]-height_off[i][:,1], label=scan_name[i]+' hor height')
#     axis[2].set_title('height (on-off)')

# for i in range(3):
#     axis[i].set_xlabel('delay (ps)', fontsize=16)
#     axis[i].set_xlim(-2, 10)
#     axis[i].legend()



# plt.savefig(path_upload+r'\trace_ds.png', format='png', dpi=300)

In [ ]:

fig, axis = plt.subplots(1,1, figsize=(7, 7),layout='tight')

end = len(delay)
c_bp = ['k', 'orange', 'g']
c_ds = ['b', 'm', 'y', 'sienna']
s = ['*-', 'o-', '+-']
leg_bp = []
leg_scan = []


# axis[0].plot(delay[o], f_x_on-f_x_on[0], label='x on')
# axis[0].plot(delay[o], f_x_off-f_x_off[0], label='x off')
for o in [0,1]:
    y = FWHM[scan_name[o]]['f_x_on']-FWHM[scan_name[o]]['f_x_off']
    axis.plot(delay[o], 1e4*cal[o]*(y-y[0]), 'o-', c=c[o], label=sname[o])
    # axis[0].errorbar(delay[o], y-y[0], yerr=FWHM[scan_name[o]]['error_x'], linestyle='-', marker='o', c=c[o], ecolor=c[o], label=sname[o])

axis.set_ylabel('$\Delta$FHWM ($10^{-4}/\AA$)')
axis.set_xlabel('delay (ps)')
axis.legend()
axis.tick_params(axis='both', direction='in', pad = 15)
axis.set_xlim(-3, 10)
axis.set_ylim(-4,1)

plt.savefig('./fit_fhwm_x.pdf', format='pdf')